In [ ]:
# https://github.com/Ko4eBHuK/nlp-23-spring/tree/main/tasks/task-03
!python --version

In [ ]:
import regex as re
import os
import nltk
from nltk.corpus import stopwords
from google.colab import drive
import pandas as pd


drive.mount('/content/gdrive')

nltk.download('stopwords')

assets_url = 'gdrive/My Drive/Colab Notebooks/nlp-2023/assets/train/'
assets_generated_url = 'gdrive/My Drive/Colab Notebooks/nlp-2023/assets/task-3/'
stops = set(stopwords.words('english'))

In [ ]:
token_freq_dict = dict()
file_names_contents_dict = dict()
S = 0

for address, dirs, files in os.walk(assets_url):
  for file_name in files:
    file_content = []
    with open(os.path.join(address, file_name), mode='r') as annotated_document_file:
      for sentence in annotated_document_file.read().split('\n\n'):
        for annotation in sentence.split('\n'):
          word_stem_lem = annotation.split('\t')
          if len(word_stem_lem) == 3:
            token = word_stem_lem[0]
            # Очистить полученные данные от знаков пунктуации. Можно использовать регулярное выражение: [^\P{P}-]+;
            if not re.match('[^\P{P}-]+', token):
              token = re.sub('[^\P{P}-]+', '', token) # Привести полученные данные к нижнему регистру;
              # Очистить полученные данные от стоп слов. Можно использовать nltk.corpus.stopwords;
              if not token in stops:
                # Словарь токенов с их частотами по всем данным
                token_freq_dict[token] = token_freq_dict.get(token, 0) + 1
                file_content.append(token)
                S += 1
    file_names_contents_dict[file_name] = file_content

In [ ]:
# save files-contents dict
with open(assets_generated_url + 'file-content-dict.tsv', 'w') as tsvfile:
        for key, value in file_names_contents_dict.items():
            tsvfile.write(f"{key}\t{value}\n")

In [ ]:
# read file-content-dict, token_freq and term-doc-matrix from files
file_names_contents_df = pd.read_table(assets_generated_url + 'file-content-dict.tsv', )
tokens_freq_df = pd.read_csv(assets_generated_url + 'token-freq.csv')

In [ ]:
term_doc_matrix_df = pd.read_csv(assets_generated_url + 'term-doc-matrix.csv', index_col=0)

In [ ]:
file_names_contents_df

In [ ]:
tokens_freq_df

In [ ]:
term_doc_matrix_df

In [ ]:
# 1. Результаты необходимо сохранить во внешние файлы в произвольном формате
  # Сохранение словаря частот токенов
import csv

token_freq_csv_file_url = assets_generated_url + 'token-freq.csv'

with open(token_freq_csv_file_url, 'w') as csvfile:
        csvfile.write('token,frequency\n')
        for key, value in token_freq_dict.items():
            csvfile.write(f"{key},{value}\n")

In [ ]:
top_2k_tokens_by_freq = dict(sorted(dict(zip(tokens_freq_df.token, tokens_freq_df.frequency)).items(), key=lambda x: x[1], reverse=True)[:2000])

In [ ]:
first_50k_docs_with_content = dict(list(dict(zip(file_names_contents_df.file_name, file_names_contents_df.content)).items())[:50000])

In [ ]:
first_50k_docs_with_content

In [ ]:
# generate term-doc matrix
term_doc_matrix = pd.DataFrame(data=0, index=first_50k_docs_with_content.keys(), columns=top_2k_tokens_by_freq.keys())

for doc_name, tokens in first_50k_docs_with_content.items():
  all_tokens = len(tokens)
  for token in tokens:
    if token in top_2k_tokens_by_freq.keys():
      term_doc_matrix[token][doc_name] += 1

# 1. Результаты необходимо сохранить во внешние файлы в произвольном формате
  # Сохранение term-doc-matrix
term_doc_matrix.to_csv(assets_generated_url + 'term-doc-matrix.csv')

In [ ]:
# 2. Реализовать один из базовых методов векторизации произвольного текста
# Разработать метод, позволяющий преобразовать произвольный текст в вектор значений tf-idf,
  # с использованием словаря наиболее частых слов и матрицы "термин-документ", полученных ранее (на шаге 1);

import math

def get_TF_vec_of_doc(doc_text, available_tokens):
  doc_tokens = re.findall(r'[^\s.!?\-;:]+', doc_text)
  TF_vec = pd.DataFrame(data=0, index=[0], columns=available_tokens)
  for doc_token in doc_tokens:
    if doc_token in available_tokens:
      TF_vec[doc_token] += 1
  for token in available_tokens:
    TF_vec[token] = TF_vec[token]/len(available_tokens)
  return TF_vec

def getIDF_vec(term_doc_matrix, available_tokens):
  IDF_vec = pd.DataFrame(data=0, index=[0], columns=available_tokens.keys())
  for token in available_tokens:
    for index, row in term_doc_matrix.iterrows():
      if row[token] != 0:
        IDF_vec[token] += 1
  for token in available_tokens:
    if IDF_vec[token][0] != 0:
      IDF_vec[token] = math.log(term_doc_matrix.shape[0] / IDF_vec[token])
  return IDF_vec

In [ ]:
get_TF_vec_of_doc('Bebkiovan shmebra wants new anime. Such  two two tow W W W W W W W W W W W W W W W W Wunicly, only it! Reks shemks gres.', top_2k_tokens_by_freq.keys())

In [ ]:
term_doc_matrix.head()

In [ ]:
idf_dataframe = getIDF_vec(term_doc_matrix_df.head(2000), top_2k_tokens_by_freq)

In [ ]:
idf_dataframe

In [ ]:
# 3. Реализовать метод, позволяющий векторизовать произвольный текст с использованием нейронных сетей (предлагается использовать стандартную реализацию модели w2v или glove). 
# Выбранную модель необходимо обучить на обучающей выборке.
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

In [ ]:
list_of_lists_in_string_representation = list(first_50k_docs_with_content.values())

In [ ]:
data = []
for list_in_string_representation in list_of_lists_in_string_representation:
  list_in_normal_form = list_in_string_representation.strip('][').split(', ')
  clear_list = []
  for word in list_in_normal_form:
    word_clear = word[1:]
    word_clear = word_clear[:-1]
    clear_list.append(word_clear)
  data.append(clear_list)

In [ ]:
data

In [ ]:
w2v_model = Word2Vec(
    min_count=10,
    vector_size=100,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

In [ ]:
w2v_model.build_vocab(data)

In [ ]:
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=5, report_delay=1)

In [ ]:
w2v_model.save(assets_generated_url + 'word2vec.model')

In [ ]:
w2v_model = Word2Vec.load(assets_generated_url + 'word2vec.model')

In [ ]:
w2v_model.wv.get_vector('walk')

In [ ]:
# 4. С использованием библиотечной реализации метода подсчета косинусного расстояния между векторными представлениями текста, 
# продемонстрировать на примерах, что для семантически близких слов модель генерирует вектора, для которых косинусное расстояние меньше, чем для семантически далеких токенов

# изменяется от 0 до 1 и чем больше - тем лсова более похожи
from scipy import spatial
def cosine_lib(a, b):
    return 1 - spatial.distance.cosine(a, b)

def cosine_similarity_of_vectors(vec_1, vec_2):
  min_vec_size = len(vec_1)
  if min_vec_size > len(vec_2):
    min_vec_size = len(vec_2)
  dot_product_of_vectors = 0
  vec_1_magnitude = 0
  vec_2_magnitude = 0
  for vec_param_index in range(0,min_vec_size):
    dot_product_of_vectors += vec_1[vec_param_index] * vec_2[vec_param_index]
    vec_1_magnitude += vec_1[vec_param_index] * vec_1[vec_param_index]
    vec_2_magnitude += vec_2[vec_param_index] * vec_2[vec_param_index]
  vec_1_magnitude = math.sqrt(vec_1_magnitude)
  vec_2_magnitude = math.sqrt(vec_2_magnitude)

  return dot_product_of_vectors / (vec_1_magnitude * vec_2_magnitude)

In [ ]:
set_of_examples = {
	'step': [['trip', 'walk'], ['count', 'destination'], ['injured', 'center']],
	'coach': [['father', 'player'], ['postgame', 'scoreboard'], ['engagement']],
	'photo': [['picture', 'image', 'painting'], ['file', 'art'], ['postgame', 'wet']],
	'ocean': [['lake', 'water', 'river'], ['fish', 'boat'], ['snap']]
}

for example_word, tests in set_of_examples.items():
  print(f"analyzing word - {example_word}")
  exmaple_word_vec = w2v_model.wv.get_vector(example_word)
  print("\tsame meaning words scores:")
  for same_meaning_word in tests[0]:
    same_meaning_word_vec = w2v_model.wv.get_vector(same_meaning_word)
    my_cos_dist = cosine_similarity_of_vectors(exmaple_word_vec, same_meaning_word_vec)
    lib_cos_dist = cosine_lib(exmaple_word_vec, same_meaning_word_vec)
    print(f"\t\tdistance for {same_meaning_word}")
    print(f"\t\t\t\tmy  = {my_cos_dist}")
    print(f"\t\t\t\tlib = {lib_cos_dist}")
  print("\tsame scope words scores:")
  for same_scope_word in tests[1]:
    same_scope_word_vec = w2v_model.wv.get_vector(same_scope_word)
    my_cos_dist = cosine_similarity_of_vectors(exmaple_word_vec, same_scope_word_vec)
    lib_cos_dist = cosine_lib(exmaple_word_vec, same_scope_word_vec)
    print(f"\t\tdistance for {same_scope_word}")
    print(f"\t\t\t\tmy  = {my_cos_dist}")
    print(f"\t\t\t\tlib = {lib_cos_dist}")
  print("\tdifferent scope words scores:")
  for diff_scope_word in tests[2]:
    diff_scope_word_vec = w2v_model.wv.get_vector(diff_scope_word)
    my_cos_dist = cosine_similarity_of_vectors(exmaple_word_vec, diff_scope_word_vec)
    lib_cos_dist = cosine_lib(exmaple_word_vec, diff_scope_word_vec)
    print(f"\t\tdistance for {diff_scope_word}")
    print(f"\t\t\t\tmy  = {my_cos_dist}")
    print(f"\t\t\t\tlib = {lib_cos_dist}")

In [ ]:
# 5. Применить какой-либо метод сокращения размерностей полученных одним из базовых способов векторизации, выбранным ранее (см. пункт 2), векторов
from sklearn.decomposition import PCA

vectors = []
df_train = pd.read_csv('gdrive/My Drive/Colab Notebooks/nlp-2023/train.csv')

for index, row in df_train.head(350).iterrows():
  text = row['title'] + '. ' + row['text']
  df_vector = get_TF_vec_of_doc(text, top_2k_tokens_by_freq.keys())
  vector_as_list = df_vector.loc[0, :].values.flatten().tolist()
  vectors.append(vector_as_list)

In [ ]:
pca = PCA(n_components=100)
pca_model = pca.fit(vectors)

In [ ]:
# 6. С использованием разработанного метода подсчета косинусного расстояния сравнить эффективность метода векторизации с использованием 
#  нейронных сетей и эффективность базовых методов векторизации с последующим сокращением размерности.
# Сформулировать вывод о том, применение какого способа позволяет получить лучшие результаты на выбранном датасете.

pca_ini_vector = pca_model.transform(get_TF_vec_of_doc('photo', top_2k_tokens_by_freq.keys()))[0]
pca_same_vector = pca_model.transform(get_TF_vec_of_doc('image', top_2k_tokens_by_freq.keys()))[0]
pca_diff_vector = pca_model.transform(get_TF_vec_of_doc('water', top_2k_tokens_by_freq.keys()))[0]

w2v_ini_vector = w2v_model.wv.get_vector('photo')
w2v_same_vector = w2v_model.wv.get_vector('image')
w2v_diff_vector = w2v_model.wv.get_vector('water')

In [ ]:
my_cos_dist_pca_same = cosine_similarity_of_vectors(pca_ini_vector, pca_same_vector)
my_cos_dist_pca_diff = cosine_similarity_of_vectors(pca_ini_vector, pca_diff_vector)

lib_cos_dist_pca_same = cosine_lib(pca_ini_vector, pca_same_vector)
lib_cos_dist_pca_diff = cosine_lib(pca_ini_vector, pca_diff_vector)


my_cos_dist_w2v_same = cosine_similarity_of_vectors(w2v_ini_vector, w2v_same_vector)
my_cos_dist_w2v_diff = cosine_similarity_of_vectors(w2v_ini_vector, w2v_diff_vector)

lib_cos_dist_w2v_same = cosine_lib(w2v_ini_vector, w2v_same_vector)
lib_cos_dist_w2v_diff = cosine_lib(w2v_ini_vector, w2v_diff_vector)

In [ ]:
print('PCA')
print('\tMY cosine')
print(f"\t\tsame={my_cos_dist_pca_same}\t\tdiff={my_cos_dist_pca_diff}")
print('\tLIB cosine')
print(f"\t\tsame={lib_cos_dist_pca_same}\t\tdiff={lib_cos_dist_pca_diff}")

print()

print('W2V')
print('\tMY cosine')
print(f"\t\tsame={my_cos_dist_w2v_same}\t\tdiff={my_cos_dist_w2v_diff}")
print('\tLIB cosine')
print(f"\t\tsame={lib_cos_dist_w2v_same}\t\tdiff={lib_cos_dist_w2v_diff}")

In [ ]:
# 7. Реализовать метод, осуществляющий векторизацию произвольного текста по следующему алгоритму: https://github.com/Ko4eBHuK/nlp-23-spring/tree/main/tasks/task-03
def vectorize_text(text):
  text_vector = [0] * 100
  total_tokens = 0
  sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s(?![a-z])', text)
  for sentence in sentences:
    tokens = re.findall(r'[^\s.!?\-;:]+', sentence)
    for token in tokens:
      if (not token in stops) and (token in w2v_model.wv.key_to_index):
        total_tokens += 1
        token_vector = w2v_model.wv.get_vector(token)
        text_vector = [sum(x) for x in zip(text_vector, token_vector)]
  return [x / total_tokens for x in text_vector]

In [ ]:
vectorize_text('Michael Phelps won the gold medal Wireless in the 400 individual medley and set a world record in a time of 4 minutes 8.26 seconds.')

In [ ]:
# 8. Выполнить векторизацию тестовой выборки с использованием метода, реализованного на предыдущем шаге.
test_url = 'gdrive/My Drive/Colab Notebooks/nlp-2023/test.csv'
df_test = pd.read_csv(test_url)

with open(f"gdrive/My Drive/Colab Notebooks/nlp-2023/assets/task-3/test-embeddings.tsv", 'w') as writefile:
  for index, row in df_test.iterrows():
    text = row['title'] + '. ' + row['text']
    doc_vector = vectorize_text(text)
    row_to_write = f"{index}"
    for embeding_component in doc_vector:
      row_to_write += f"\t{embeding_component}"
    writefile.write(row_to_write + '\n')